In [23]:
import os
import numpy as np
import pandas as pd

In [24]:
%pwd

'/Users/tapankhaladkar/Natural Language Processing/Text-Summarizer/NLP-Text-Summarizer'

In [12]:
#os.chdir('../')

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class data_ingestionConfig:
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    root_dir: Path = Path


In [26]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [16]:
class ConfigManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> data_ingestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = data_ingestionConfig(
            source_url=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
            root_dir=Path(config.root_dir)
        )

        return data_ingestion_config

In [17]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self, config: data_ingestionConfig):
        self.config = config


    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(self.config.source_url, self.config.local_data_file)
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}") 

    def extract_zip_file(self):
        if not os.path.exists(self.config.unzip_dir):
            os.makedirs(self.config.unzip_dir)
            logger.info(f"Created directory: {self.config.unzip_dir}")

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
            logger.info(f"Extracted files to {self.config.unzip_dir}")

In [29]:
try:
    config_manager = ConfigManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_data()
    data_ingestion.extract_zip_file()

except Exception as e:
    logger.exception(f"An error occurred during data ingestion: {e}")
    raise e

[2025-08-06 19:46:05,593: INFO: YAML file config/config.yaml loaded successfully.]
[2025-08-06 19:46:05,599: INFO: YAML file params.yaml loaded successfully.]
[2025-08-06 19:46:05,603: INFO: Directory created at: artifacts.]
[2025-08-06 19:46:05,604: INFO: Directory created at: artifacts/data_ingestion.]
[2025-08-06 19:46:06,946: INFO: artifacts/data_ingestion/summarizer-data.zip downloaded! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F8BD:135C65:188EE3:20E118:6893E93D
Accept-Ranges: bytes
Date: Wed, 06 Aug 2025 23:46:06 GMT
Via: 1.1 varnish
X-Served-By: cache-nyc-kteb1890071-NYC
X-Cache: MISS
X-Cache-Hits: 0
X-

In [30]:
#from src.textsummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

#print("CONFIG_FILE_PATH:", CONFIG_FILE_PATH)
#print("CONFIG_FILE_PATH exists:", CONFIG_FILE_PATH.exists())
#print("PARAMS_FILE_PATH:", PARAMS_FILE_PATH)
#print("PARAMS_FILE_PATH exists:", PARAMS_FILE_PATH.exists())